In [ ]:
from torch.nn.utils.rnn import pad_sequence
import torch
a = torch.ones(25, 310)
b = torch.ones(25, 320)
c = torch.ones(25, 330)
torch.cat((a, b, c), dim=1).shape


In [ ]:
import torch
from merge_raw_data_and_features import get_data_by_feature_type,get_row_data
from torch.nn.utils.rnn import pad_sequence
data_row_all = torch.zeros(15,3,15,62,53001)
for sub in range(15):
    for session in range(3):
        temp_exp_data = []
        for exp in range(15):
            temp_data,_ = get_row_data(sub+1,session+1,exp+1)
            if temp_data.shape[1] < 53001:
                # Calculate the amount of padding needed
                padding_size = 53001 - temp_data.shape[1] 
                
                # Pad the tensor with zeros on the second dimension (at the end)
                eeg_data_padded = torch.nn.functional.pad(temp_data, (0, padding_size), "constant", 0)
                temp_exp_data.append(eeg_data_padded)
            else:
                # If no padding is needed, add the tensor as it is
                temp_exp_data.append(temp_data)
                print(temp_data.shape)
            print(sub+1,session+1,exp+1,"is done")
        data_row_all[sub,session] = torch.stack(temp_exp_data)
print(data_row_all)

In [ ]:
torch.save(data_row_all,"/home/wf/EEG_GTN/data/dataset/SEED/Preprocessed_EEG/data_all.pt")

In [ ]:
import torch
import os
features=["de","psd","dasm","rasm","dcau"]
seed_coh_adj = torch.load("/home/wf/EEG_GTN/data/dataset/SEED/adjacency_matrix/all_coh_adj.pt")
seed_nmi_adj = torch.load("/home/wf/EEG_GTN/data/dataset/SEED/adjacency_matrix/all_nmi_adj.pt")
seed_pcc_adj = torch.load("/home/wf/EEG_GTN/data/dataset/SEED/adjacency_matrix/all_pcc_adj.pt")
seed_plv_adj = torch.load("/home/wf/EEG_GTN/data/dataset/SEED/adjacency_matrix/all_plv_adj.pt")
for feature in features:
    seed_data_dir = f"/home/wf/EEG_GTN/data/dataset/Processed_Dataset/seed/zero/{feature}/data.pt"

    seed_data = torch.load(seed_data_dir)
    print(seed_data.shape)

    seed_conbian_data_all = []
    for subject in range(15):
        seed_conbian_data_subject = []
        for session in range(3):
            seed_conbian_data_session = []
            for exp in range(15):
                seed_conbian_data_session.append({
                    "data": seed_data[subject,session,exp],
                    "coh_adj": seed_coh_adj[subject,session,exp],
                    "nmi_adj": seed_nmi_adj[subject,session,exp],
                    "pcc_adj": seed_pcc_adj[subject,session,exp],
                    "plv_adj": seed_plv_adj[subject,session,exp]
                    })
                print(feature,subject,session,exp,"is done")
            seed_conbian_data_subject.append(seed_conbian_data_session)
        seed_conbian_data_all.append(seed_conbian_data_subject)
                
    save_dir = f"/home/wf/EEG_GTN/data/dataset/Processed_Dataset/seed/zero_with_adj/{feature}"
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    torch.save(seed_conbian_data_all, f"/home/wf/EEG_GTN/data/dataset/Processed_Dataset/seed/zero_with_adj/{feature}/data.pt")

# SEED IV

In [ ]:
import os
import re
import scipy.io as scio
import numpy as np
import torch
from utils.adj_utils import *
# 从data.keys()中获取 key
def get_trail_keys(data_keys):
    s=str(data_keys)

    keys = re.findall(r"'(.*?)'", s)[3:]
    return keys

session1_label = [1,2,3,0,2,0,0,1,0,1,2,1,1,1,2,3,2,2,3,3,0,3,0,3]
session2_label = [2,1,3,0,0,2,0,2,3,3,2,3,2,0,1,1,2,1,0,3,0,1,3,1]
session3_label = [1,2,2,1,3,3,3,1,1,2,1,0,2,3,3,0,2,3,0,0,2,0,1,0]
session_label=[session1_label,session2_label,session3_label]

root = "/home/wf/EEG_GTN/data/dataset/SEED_IV/eeg_raw_data"
all_pcc_adj = torch.zeros((15, 3, 24, 62, 62))
all_nmi_adj = torch.zeros((15, 3, 24, 62, 62))
all_coh_adj = torch.zeros((15, 3, 24, 62, 62))
all_plv_adj = torch.zeros((15, 3, 24, 62, 62))
all_label = torch.zeros((15, 3, 24,2))
session_dirs=os.listdir(root)
session_dirs.sort()
for session in range(3):
    sub_file_names=os.listdir(os.path.join(root,str(session+1)))
    sub_file_names.sort()
    for subject, sub_file_name in enumerate(sub_file_names):
        data = scio.loadmat(os.path.join(root, str(session+1), sub_file_name))
        keys=get_trail_keys(data.keys())
        for exp, trail_index in enumerate(keys):
            data_trail=data[trail_index]  # (62,3xxxx)
            all_label[subject, session, exp, 0] = torch.from_numpy(
                np.array(session_label[session][exp]))
            all_label[subject, session, exp, 1] = torch.from_numpy(
                np.array(subject))
            # all_pcc_adj[subject, session, exp] = torch.from_numpy(
            #     cal_pcc_matrix(data_trail))
            # all_nmi_adj[subject, session, exp] = torch.from_numpy(
            #     cal_nmi_matrix(data_trail))
            # all_coh_adj[subject, session, exp] = torch.from_numpy(
            #     cal_coherence_matrix(data_trail))
            # all_plv_adj[subject, session, exp] = torch.from_numpy(
            #     cal_plv_matrix(data_trail))
           
            print(
                f"subject: {subject+1}, session: {session+1}, exp: {exp+1}", "is done.")
        

In [4]:
torch.save(all_pcc_adj,
           "/home/wf/EEG_GTN/data/dataset/SEED_IV/adjacency_matrix/all_pcc_adj.pt")
torch.save(all_nmi_adj,
           "/home/wf/EEG_GTN/data/dataset/SEED_IV/adjacency_matrix/all_nmi_adj.pt")
torch.save(all_coh_adj,
           "/home/wf/EEG_GTN/data/dataset/SEED_IV/adjacency_matrix/all_coh_adj.pt")
torch.save(all_plv_adj,
           "/home/wf/EEG_GTN/data/dataset/SEED_IV/adjacency_matrix/all_plv_adj.pt")

In [2]:
torch.save(all_label,"/home/wf/EEG_GTN/data/dataset/Processed_Dataset/seed_iv/zero_with_adj/de/label.pt")

In [ ]:
def linear_interpolate(data, max_length):
    interpolated_data = []

    for trial in data:
        current_length = trial.shape[1]
        # 如果当前长度已经等于最大长度，则无需插值
        if current_length == max_length:
            interpolated_data.append(trial)
            continue

        # 计算插值步数
        steps = max_length - current_length

        # 生成插值索引
        interpolate_indices = np.linspace(
            0, current_length, num=max_length, endpoint=False)

        # 初始化插值后的trial
        interpolated_trial = torch.zeros(
            (trial.shape[0], max_length, trial.shape[2]))

        prev_idx = 0
        for new_idx, interp_idx in enumerate(interpolate_indices):
            # 向下取整以获取当前索引
            current_idx = int(np.floor(interp_idx))

            # 插值
            if current_idx == prev_idx:
                interpolated_trial[:, new_idx, :] = trial[:, current_idx, :]
            else:
                # 计算两个相邻点的均值作为插值结果
                interpolated_trial[:, new_idx, :] = (
                    trial[:, current_idx, :] + trial[:, prev_idx, :]) / 2
                prev_idx = current_idx

        interpolated_data.append(interpolated_trial)

    return interpolated_data

def pad_trials(data):
    # 确定最大长度
    max_length = 64

    # 初始化一个空列表来存储填充后的数据
    padded_data = []

    # 遍历所有trial
    for trial in data:
        # 当前trial的长度
        current_length = trial.shape[1]

        # 如果当前trial长度小于最大长度，则进行填充
        if current_length < max_length:
            # 计算需要填充的长度
            padding_size = max_length - current_length

            # 创建一个与当前trial相同形状的全0 tensor进行填充
            padding = torch.zeros((trial.shape[0], padding_size, trial.shape[2]))

            # 将原始数据和填充部分拼接起来
            padded_trial = torch.cat((trial, padding), dim=1)
        else:
            padded_trial = trial

        # 将填充后的trial添加到列表中
        padded_data.append(padded_trial)

    return padded_data

data_save_zero = torch.zeros((15,3,24,62,64,5))
seed_data_dir = "/home/wf/EEG_GTN/data/dataset/Processed_Dataset/seed_iv/list/de/data.pt"
seed_data = torch.load(seed_data_dir)
for subject in range(15):
        for session in range(3):
            data_list = seed_data[subject][session]
            # data_list = linear_interpolate(data_list,64)
            data_list = pad_trials(data_list)
            for exp in range(24):
                data_save_zero[subject, session, exp] = data_list[exp]
                print(subject,session,exp,"is done")

In [12]:
torch.save(data_save_zero,"/home/wf/EEG_GTN/data/dataset/Processed_Dataset/seed_iv/zero/de/data.pt")

In [13]:
import torch
import os
features = ["de", "psd"]
seed_coh_adj = torch.load(
    "/home/wf/EEG_GTN/data/dataset/SEED_IV/adjacency_matrix/all_coh_adj.pt")
seed_nmi_adj = torch.load(
    "/home/wf/EEG_GTN/data/dataset/SEED_IV/adjacency_matrix/all_nmi_adj.pt")
seed_pcc_adj = torch.load(
    "/home/wf/EEG_GTN/data/dataset/SEED_IV/adjacency_matrix/all_pcc_adj.pt")
seed_plv_adj = torch.load(
    "/home/wf/EEG_GTN/data/dataset/SEED_IV/adjacency_matrix/all_plv_adj.pt")
for feature in features:
    seed_data_dir = f"/home/wf/EEG_GTN/data/dataset/Processed_Dataset/seed_iv/zero/{feature}/data.pt"

    seed_data = torch.load(seed_data_dir)

    seed_conbian_data_all = []
    for subject in range(15):
        seed_conbian_data_subject = []
        for session in range(3):
            seed_conbian_data_session = []
            for exp in range(24):
                seed_conbian_data_session.append({
                    "data": seed_data[subject, session, exp],
                    "coh_adj": seed_coh_adj[subject, session, exp],
                    "nmi_adj": seed_nmi_adj[subject, session, exp],
                    "pcc_adj": seed_pcc_adj[subject, session, exp],
                    "plv_adj": seed_plv_adj[subject, session, exp]
                })
                print(feature, subject, session, exp, "is done")
            seed_conbian_data_subject.append(seed_conbian_data_session)
        seed_conbian_data_all.append(seed_conbian_data_subject)

    save_dir = f"/home/wf/EEG_GTN/data/dataset/Processed_Dataset/seed_iv/zero_with_adj/{feature}"
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    torch.save(seed_conbian_data_all,
               f"/home/wf/EEG_GTN/data/dataset/Processed_Dataset/seed_iv/zero_with_adj/{feature}/data.pt")

de 0 0 0 is done
de 0 0 1 is done
de 0 0 2 is done
de 0 0 3 is done
de 0 0 4 is done
de 0 0 5 is done
de 0 0 6 is done
de 0 0 7 is done
de 0 0 8 is done
de 0 0 9 is done
de 0 0 10 is done
de 0 0 11 is done
de 0 0 12 is done
de 0 0 13 is done
de 0 0 14 is done
de 0 0 15 is done
de 0 0 16 is done
de 0 0 17 is done
de 0 0 18 is done
de 0 0 19 is done
de 0 0 20 is done
de 0 0 21 is done
de 0 0 22 is done
de 0 0 23 is done
de 0 1 0 is done
de 0 1 1 is done
de 0 1 2 is done
de 0 1 3 is done
de 0 1 4 is done
de 0 1 5 is done
de 0 1 6 is done
de 0 1 7 is done
de 0 1 8 is done
de 0 1 9 is done
de 0 1 10 is done
de 0 1 11 is done
de 0 1 12 is done
de 0 1 13 is done
de 0 1 14 is done
de 0 1 15 is done
de 0 1 16 is done
de 0 1 17 is done
de 0 1 18 is done
de 0 1 19 is done
de 0 1 20 is done
de 0 1 21 is done
de 0 1 22 is done
de 0 1 23 is done
de 0 2 0 is done
de 0 2 1 is done
de 0 2 2 is done
de 0 2 3 is done
de 0 2 4 is done
de 0 2 5 is done
de 0 2 6 is done
de 0 2 7 is done
de 0 2 8 is done
de 

In [8]:
import torch
data = torch.load(
    "/home/wf/EEG_GTN/data/dataset/Processed_Dataset/seed_iv/zero_with_adj/de/data.pt")
print(data[0][0][0]['data'].shape)

torch.Size([62, 64, 5])


In [10]:
import numpy as np
z = np.array([1, 1, 0.1, 0.1, 1])
print(np.exp(z)/sum(np.exp(z)))
# [0.26768323 0.26768323 0.09847516 0.09847516 0.26768323]
# [0.26768323 0.26768323 0.09847516 0.09847516 0.26768323]

[0.2622511  0.2622511  0.10662334 0.10662334 0.2622511 ]


In [1]:
import subprocess

# Loop through bands from 0 to 4 and execute the command
for feature in ['dasm', 'rasm','dcau']:
    for band in range(5):
        command = f'python -u "/home/wf/EEG_GTN/main.py" --dataset seed_adj --train_mode si --batch_size 16 --bands {band} --feature_type {feature}'
        print(f"Executing: {command}")
        subprocess.run(command, shell=True)

Executing: python -u "/home/wf/EEG_GTN/main.py" --dataset seed_adj --train_mode si --batch_size 16 --bands 0 --feature_type psd
sub0 is start:
features length: 265     channels nums: 62
Namespace(adj_num=5, attention_out=265, bands=0, batch_size=16, channels_num=62, clip_length=1, config_path='/home/wf/EEG_GTN/global.config', cur_exp_index=0, cur_session_index=0, cur_sub_index=0, dataset='seed_adj', device='cuda:0', domain_class=15, dropout=0.5, epochs=200, feature_len=265, feature_type='psd', gpu=0, graph_out=128, graph_readout_dim=64, grl_alpha=1.0, k=10, kadj=3, label_type='valence', loss1=5e-06, loss2=0.0001, loss_beta=1.0, lr=0.0001, model='MAGCN_func', nclass=3, num_levels=3, raw_len=53001, save_model=0, save_path='./data/parser_save/seed_adj/20231229_07:43:02', save_tsne_cm=0, se_squeeze_ratio=4, seed=3407, session=0, split_method='k_fold', spp=True, train_mode='si')
Validation Results - Epoch: 0 acc: 0.4375 loss: 33.3493
Validation Results - Epoch: 1 acc: 0.5000 loss: 26.1512
V